In [31]:
from utils import query_raven, build_raven_prompt
import requests

In [32]:
def get_age(age: int = None) -> int:
    """
    Validates and returns the age value.

    Args:
        age (int, optional): The age of the client. Must be a non-negative integer. If not provided, defaults to None.

    Returns:
        int: The validated age. If no age is provided, None is returned.

    Raises:
        ValueError: If `age` is provided and is a negative integer.
    """
    if age is not None and age < 0:
        raise ValueError("age must be a non-negative integer.")

    return age

In [33]:
def get_gender(gender: str = None) -> str:
    """
    Will return if the client is male or not

    Args:
        gender (str, optional): the gender of the client. Can only be one of the following: 'Male', 'Female'

    Returns:
        str: The gender of the client.

    Raises:
        ValueError: If `gender` is not one of the valid options.
    """

    if gender is not None and gender not in ["Male", "Female"]:
        raise ValueError("Gender not given")

    return gender

In [34]:
def get_most_frequent_sector_name(most_frequent_sector_name: str = None) -> str:
    """
    Will return the most frequent sector that the clients orders are in.

    Args:
        most_frequent_sector_name (str, optional): The sector in which the client works. Must match exactly one of the following sectors:
                                     ["Financial", "Industrial", "Technology"].

    Returns:
        str: The sector name if valid. If no sector name is provided, None is returned.

    Raises:
        ValueError: If `most_frequent_sector_name` is not one of the specified values.
    """
    valid_sectors = ["Financials", "Industries"]
    if most_frequent_sector_name and most_frequent_sector_name not in valid_sectors:
        raise ValueError(f"Please provide a valid sector: {', '.join(valid_sectors)}")
    return most_frequent_sector_name

In [35]:
def get_risk_rate(risk_rate: str = None) -> str:
    """
    Will return the risk rate of the client

    Args:
        risk_rate (str, optional): The risk rate of the client. Can only be one of the following: 'High', 'Low', 'Medium', 'Not Assigned'

    Returns:
        str: A valid category for the risk rate of the client

    Raises:
        ValueError: If `risk_rate` is not one of the valid options.

    """
    if risk_rate is not None and risk_rate not in [
        "High",
        "Low",
        "Medium",
        "Not Assigned",
    ]:
        raise ValueError("Risk rate is invalid")

    return risk_rate

In [36]:
def get_completed_orders(completed_orders: str = None) -> str:
    """
    Will return the completed orders of the client

    Args:
        completed_orders (str, optional): The completed orders of the client. can only be one of the following:
        'All', 'More Than Half', 'Less Than Half', 'None'

    Returns:
        str: A valid category that shows the trend of the clients completed orders

    Raises:
        ValueError: If `completed_orders` is not one of the valid options.
    """
    if completed_orders and completed_orders not in [
        "All",
        "More Than Half",
        "Less Than Half",
        "None",
    ]:
        raise ValueError("Completed orders ratio is invalid")

    return completed_orders

In [37]:
def get_canceled_orders(canceled_orders: str = None) -> str:
    """
    Will return the canceled orders of the client

    Args:
        canceled_orders (str, optional): The canceled orders of the client. Can only be one of the following:
        'All', 'Most', 'Moderate', 'Little', 'None'

    Returns:
        str: A valid category that shows the trend of the clients canceled orders

    Raises:
        ValueError: If `canceled_orders` is not one of the valid options.
    """
    if canceled_orders and canceled_orders not in [
        "All",
        "Most",
        "Moderate",
        "Little",
        "None",
    ]:
        raise ValueError("Canceled orders ratio is invalid")

    return canceled_orders

In [38]:
def get_average_price(average_price: float = None) -> float:
    """
    Will return the average price of the clients orders

    Args:
        average_price (int, optional): The average price of the clients orders

    Returns:
        float: The validated average price of the clients orders

    Raises:
        ValueError: If `average_price` is a negative value
    """
    if average_price and average_price < 0:
        raise ValueError("Average price cannot be negative")

    return average_price

In [39]:
def get_most_frequent_order_type(most_frequent_order_type: str = None) -> str:
    """
    Will return the most frequent order type of the client

    Args:
        most_frequent_order_type (str, optional): The most frequent order type of the client. Can only be one of the following:
        'Buy', 'Sell'

    Returns:
        str: A valid category for the most frequent order type of the client

    Raises:
        ValueError: If `most_frequent_order_type` is not one of the valid options.
    """
    if most_frequent_order_type and most_frequent_order_type not in ["Buy", "Sell"]:
        raise ValueError("Most frequent order type is invalid")

    return most_frequent_order_type

In [40]:
def get_most_frequent_execution_status(
    most_frequent_execution_status: str = None,
) -> str:
    """
    Will return the most frequent execution status of the clients orders

    Args:
        most_frequent_execution_status (str, optional): The most frequent execution status of the clients orders.
        Can only be one of the following: 'Executed', 'Not Executed', 'Partially Executed'

    Returns:
        str: A valid category for the most frequent execution status of the clients orders

    Raises:
        ValueError: If `most_frequent_execution_status` is not one of the valid options.
    """
    if most_frequent_execution_status and most_frequent_execution_status not in [
        "Executed",
        "Not Executed",
        "Partially Executed",
    ]:
        raise ValueError("Most frequent execution status is invalid")

    return most_frequent_execution_status

In [41]:
def get_avg_order_rate_difference(avg_order_rate_difference: str = None) -> str:
    """
    Will return the change in the client's order activity

    Args:
        avg_order_rate_difference (str, optional): The change in the client's order activity.
        Can only be one of the following: 'increased', 'decreased', 'constant'

    Returns:
        str: A valid category for the change in the client's order activity

    Raises:
        ValueError: If `avg_order_rate__difference` is not one of the valid options.
    """
    if avg_order_rate_difference and avg_order_rate_difference not in [
        "increased",
        "decreased",
        "constant",
    ]:
        raise ValueError("Average order rate difference is invalid")

    return avg_order_rate_difference

In [42]:
def get_avg_order_quantity_rate_difference(
    avg_order_quantity_rate_difference: str = None,
) -> str:
    """
    Will return the change in the client's order quantity

    Args:
        avg_order_quantity_rate_difference (str, optional): The change in the client's order quantity.
        Can only be one of the following: 'increased', 'decreased', 'constant'

    Returns:
        str: A valid category for the change in the client's order quantity

    Raises:
        ValueError: If `avg_order_quantity_rate__difference` is not one of the valid options.
    """
    if (
        avg_order_quantity_rate_difference
        and avg_order_quantity_rate_difference
        not in [
            "increased",
            "decreased",
            "constant",
        ]
    ):
        raise ValueError("Average order quantity rate difference is invalid")
    return avg_order_quantity_rate_difference

In [43]:
def handle_unspecified_client_data(missing_client_data: list[str]) -> str:
    """
    Handles the case where client details are not provided by the user.

    Args:
        missing_client_data (list[str]): The list of the missing client details.

    Returns:
        str: A message prompting the user to provide the missing arguments.
    """
    return (
        f"Please provide the following missing values: {', '.join(missing_client_data)}"
    )

In [44]:
def no_relevant_function(prompt: str) -> dict:
    """
    Call this when no other provided function can be called to answer the user query.

    Args:
       prompt: The prompt that cannot be answered by any other function calls.
    """
    no_function_calling_prompt = f"""
    <s> [INST] {prompt} [/INST]
    <s> [INST] I am called Raven. How can i assist you today?[/INST]
    """
    return {"message": query_raven(no_function_calling_prompt)}

In [45]:
def unknown_arguments(unknown_arg: str) -> str | int:
    """
    Provides default values for unknown or unspecified client details in a user query.

    This function returns a default value for specific client details that are not known
    or provided by the user.

    Args:
        unknown_arg (str): The name of the unknown or unspecified client detail. Could be one of the following:
        [
            age,
            risk_rate,
            gender,
            completed_orders_ratio,
            canceled_orders_ratio,
            avg_price,
            most_frequent_order_type,
            most_frequent_execution_status,
            most_frequent_sector_name,
            avg_order_rate__difference,
            avg_order_quantity_rate__difference
        ]

    Returns:
        str: The default value corresponding to the provided unknown client detail.
    """
    if unknown_arg == "age":
        return 40
    if unknown_arg == "gender":
        return "Male"
    if unknown_arg == "most_frequent_sector_name":
        return "Financials"
    if unknown_arg == "risk_rate":
        return "Not Assigned"
    if unknown_arg == "completed_orders_ratio":
        return "Most"
    if unknown_arg == "canceled_orders_ratio":
        return "Moderate"
    if unknown_arg == "avg_price":
        return 9.56
    if unknown_arg == "most_frequent_order_type":
        return "Sell"
    if unknown_arg == "most_frequent_execution_status":
        return "Executed"
    if unknown_arg == "avg_order_rate_difference":
        return "constant"
    if unknown_arg == "avg_order_quantity_rate_difference":
        return "constant"

In [46]:
def construct_client_dict(
    age: int = None,
    risk_rate: str = None,
    gender: str = None,
    completed_orders: str = None,
    canceled_orders: str = None,
    average_price: float = None,
    most_frequent_order_type: str = None,
    most_frequent_execution_status: str = None,
    most_frequent_sector_name: str = None,
    avg_order_rate_difference: str = None,
    avg_order_quantity_rate_difference: str = None,
) -> dict:
    """
    Constructs a dict with all the client details specified in the user query.

    Args:
        age (int): The age of the client.
        risk_rate (str): The risk rate of the client.
        gender (str): The gender of the client.
        completed_orders (str): A valid category that shows the trend of the clients completed orders
        canceled_orders_ratio (str): A valid category that shows the trend of the clients canceled orders
        average_price (float): The validated average price of the clients orders
        most_frequent_order_type (str): The most frequent order type of the client.
        most_frequent_execution_status (str): The most frequent execution status of the clients orders.
        most_frequent_sector_name (str): The most frequent sector that the clients orders are in.
        avg_order_rate_difference (str): The change in the client's order activity.
        avg_order_quantity_rate_difference (str): The change in the client's order quantity.

    Returns:
        dict: The client data.
    """

    provided_params = locals().copy()

    # try:
    #     provided_params["age"] = get_age(age)
    #     provided_params["gender"] = get_gender(gender)
    #     provided_params["risk_rate"] = get_risk_rate(risk_rate)
    #     provided_params["completed_orders"] = get_completed_orders(completed_orders)
    #     provided_params["canceled_orders"] = get_canceled_orders(canceled_orders)
    #     provided_params["average_price"] = get_average_price(average_price)
    #     provided_params["most_frequent_order_type"] = get_most_frequent_order_type(
    #         most_frequent_order_type
    #     )
    #     provided_params["most_frequent_execution_status"] = (
    #         get_most_frequent_execution_status(most_frequent_execution_status)
    #     )
    #     provided_params["most_frequent_sector_name"] = get_most_frequent_sector_name(
    #         most_frequent_sector_name
    #     )
    #     provided_params["avg_order_rate_difference"] = get_avg_order_rate_difference(
    #         avg_order_rate_difference
    #     )
    #     provided_params["avg_order_quantity_rate_difference"] = (
    #         get_avg_order_quantity_rate_difference(avg_order_quantity_rate_difference)
    #     )

    # except ValueError as e:
    #     return {"client_data": provided_params, "message": e.args[0]}

    message = ""
    missing_args = [key for key, value in provided_params.items() if value is None]
    if missing_args:
        message = handle_unspecified_client_data(missing_args)

    return {"client_data": provided_params, "message": message}

In [47]:
class Chatbot:
    def __init__(self):
        self.history = {}

    def add_to_history(self, bot_response: dict):
        if "client_data" in bot_response.keys():
            self.history = bot_response

    def generate_prompt(self, user_input: str) -> str:
        return build_raven_prompt(
            [
                no_relevant_function,
                construct_client_dict,
                unknown_arguments,
                get_age,
                get_risk_rate,
                get_gender,
                get_completed_orders,
                get_canceled_orders,
                get_average_price,
                get_most_frequent_order_type,
                get_most_frequent_execution_status,
                get_most_frequent_sector_name,
                get_avg_order_rate_difference,
                get_avg_order_quantity_rate_difference,
            ],
            f"{self.history}\n{user_input}",
        )

    def get_response(self, prompt: str) -> dict:
        func = query_raven(prompt)
        if func is None:
            return {"message": "Sorry, I couldn't understand your request."}

        try:
            print(func)
            response = eval(func)

            return response
        except Exception as e:
            return {"message": e}

    def predict_churn(self, client_data: dict) -> str:
        print(client_data)
        self.history = {}
        return "The client will churn"

    def handle_input(self, user_input: str) -> dict:
        prompt = self.generate_prompt(user_input)
        response = self.get_response(prompt)
        self.add_to_history(response)
        if response["message"] == "":
            response["message"] = self.predict_churn(response["client_data"])
        return response

In [58]:
# Initialize the chatbot
def run(USER_QUERY):
    chatbot = Chatbot()

    user_input = USER_QUERY
    
    print(f"user input: {user_input}")
    response = chatbot.handle_input(user_input)
    print(f"Bot: {response["message"]}")

In [63]:
USER_QUERY = "The client is a 19 year old woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered."
run(USER_QUERY)

user input: The client is a 19 year old woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered.
construct_client_dict(age=get_age(age=19), risk_rate=get_risk_rate(risk_rate='Low'), gender=get_gender(gender='Female'), completed_orders=get_completed_orders(completed_orders='All'), canceled_orders=get_canceled_orders(canceled_orders='None'), average_price=get_average_price(average_price=7.65), most_frequent_order_type=get_most_frequent_order_type(most_frequent_order_type='Buy'), most_frequent_execution_status=get_most_frequent_execution_status(most_frequent_execution_status='Executed'), most_frequent_sector_name=get_most_frequent_sector_name(most_frequent_sector_name='Industries'), avg_order_rate_difference=get_avg_order_rate_difference(avg_order_rate_difference='increased'), avg_order_quantity_rate_difference=get_avg_order_

In [64]:
USER_QUERY = "The client is a woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered. Most of the client's orders are completed and little of the client's orders are canceled."
run(USER_QUERY)

user input: The client is a woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered. Most of the client's orders are completed and little of the client's orders are canceled.
construct_client_dict(gender='Female', risk_rate='Low', most_frequent_order_type='Buy', most_frequent_execution_status='Executed', most_frequent_sector_name='Industries', avg_order_rate_difference='increased', avg_order_quantity_rate_difference='decreased')
Bot: Please provide the following missing values: age, completed_orders, canceled_orders, average_price


In [65]:
USER_QUERY = "The client is a 19 year old woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered. Most of the client's orders are completed and little of the client's orders are canceled."
run(USER_QUERY)

user input: The client is a 19 year old woman with a low risk rate. The client mainly buys stocks in the Industries sector with an average price of 7.65 and the orders are usually fully executed. They have increased order activity but decreased quantity ordered. Most of the client's orders are completed and little of the client's orders are canceled.
construct_client_dict(age=get_age(age=19), risk_rate=get_risk_rate(risk_rate='Low'), gender=get_gender(gender='Female'), completed_orders=get_completed_orders(completed_orders='All'), canceled_orders=get_canceled_orders(canceled_orders='Little'), average_price=get_average_price(average_price=7.65), most_frequent_order_type=get_most_frequent_order_type(most_frequent_order_type='Buy'), most_frequent_execution_status=get_most_frequent_execution_status(most_frequent_execution_status='Executed'), most_frequent_sector_name=get_most_frequent_sector_name(most_frequent_sector_name='Industries'), avg_order_rate_difference=get_avg_order_rate_differen

In [68]:
chatbot = Chatbot()

user_input = ""

while user_input != "exit":
    user_input = input("User: ")
    print(f"user input: {user_input}")
    response = chatbot.handle_input(user_input)
    print(f"Bot: {response["message"]}")

user input: the client is a woman
construct_client_dict(gender=get_gender())
Bot: Please provide the following missing values: age, risk_rate, gender, completed_orders, canceled_orders, average_price, most_frequent_order_type, most_frequent_execution_status, most_frequent_sector_name, avg_order_rate_difference, avg_order_quantity_rate_difference
user input: i dont know anything other than the client is a eoman
construct_client_dict(gender='Female')
Bot: Please provide the following missing values: age, risk_rate, completed_orders, canceled_orders, average_price, most_frequent_order_type, most_frequent_execution_status, most_frequent_sector_name, avg_order_rate_difference, avg_order_quantity_rate_difference
user input: i dont know
construct_client_dict(gender='Female')
Bot: Please provide the following missing values: age, risk_rate, completed_orders, canceled_orders, average_price, most_frequent_order_type, most_frequent_execution_status, most_frequent_sector_name, avg_order_rate_diffe

In [69]:
chatbot = Chatbot()

user_input = ""

while user_input != "exit":
    user_input = input("User: ")
    print(f"user input: {user_input}")
    response = chatbot.handle_input(user_input)
    print(f"Bot: {response["message"]}")

user input: i dont know anthing baout the client but i want you to put all the defaults
construct_client_dict(age=unknown_arguments(unknown_arg='age'), risk_rate=unknown_arguments(unknown_arg='risk_rate'), gender=unknown_arguments(unknown_arg='gender'), completed_orders=unknown_arguments(unknown_arg='completed_orders'), canceled_orders=unknown_arguments(unknown_arg='canceled_orders'), average_price=unknown_arguments(unknown_arg='average_price'), most_frequent_order_type=unknown_arguments(unknown_arg='most_frequent_order_type'), most_frequent_execution_status=unknown_arguments(unknown_arg='most_frequent_execution_status'), most_frequent_sector_name=unknown_arguments(unknown_arg='most_frequent_sector_name'), avg_order_rate_difference=unknown_arguments(unknown_arg='avg_order_rate_difference'), avg_order_quantity_rate_difference=unknown_arguments(unknown_arg='avg_order_quantity_rate_difference'))
Bot: Please provide the following missing values: completed_orders, canceled_orders, average_p

In [71]:
chatbot = Chatbot()

user_input = ""

while user_input != "exit":
    user_input = input("User: ")
    print(f"user input: {user_input}")
    response = chatbot.handle_input(user_input)
    print(f"Bot: {response["message"]}")

user input: the client is a woman
construct_client_dict(gender=get_gender())
Bot: Please provide the following missing values: age, risk_rate, gender, completed_orders, canceled_orders, average_price, most_frequent_order_type, most_frequent_execution_status, most_frequent_sector_name, avg_order_rate_difference, avg_order_quantity_rate_difference
user input: shes 16
construct_client_dict(age=get_age(age=16), risk_rate=get_risk_rate(risk_rate='High'), gender=get_gender(gender='Female'), completed_orders=get_completed_orders(completed_orders='All'), canceled_orders=get_canceled_orders(canceled_orders='All'), average_price=get_average_price(average_price=100), most_frequent_order_type=get_most_frequent_order_type(most_frequent_order_type='Buy'), most_frequent_execution_status=get_most_frequent_execution_status(most_frequent_execution_status='Executed'), most_frequent_sector_name=get_most_frequent_sector_name(most_frequent_sector_name='Financial'), avg_order_rate_difference=get_avg_order_ra